In [1]:
import json
import requests
import pandas as pd 
from snowflake.connector import connect 
from snowflake.connector.pandas_tools import write_pandas  
from dotenv import load_dotenv 
import os

# Data Extraction

In [2]:
url = 'https://newsapi.org/v2/everything?'  # url of the api

In [3]:
# def data_extract(file):
#     try: 
#         data = requests.get(url, params = json.loads(open(file, 'r').read())).json()
#         return data
#     except PathError:
#         print('Check your input path')

In [4]:
def data_extract(file):
    try:
        try:
            file_open = open(file, 'r').read()   # file here is basically the path of your file containing the payload
            file_open = json.loads(file_open)
            print('File loaded successfully!')
        except:
            print('Error in your configuration file')
        
        file_open['apiKey'] = os.getenv('apiKey')
        
        data = requests.get(url, params = file_open).json()
        print('Data extracted successfully!')
        return data
    except:
        print('check your configuration path')

In [5]:
extracted_data = data_extract(r'C:\Users\JayashreeArunan\Desktop\Python\Data_extraction\configuration\api_config.json')

File loaded successfully!
Data extracted successfully!


In [6]:
df = pd.DataFrame(extracted_data['articles'])
df.head(5)

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Yahoo Entertainment'}",Cristina Alexander,Overwatch 2 shows off Transformers skins in an...,Blizzard and Hasbro have revealed the Transfor...,https://consent.yahoo.com/v2/collectConsent?se...,None,2024-07-08T22:23:59Z,"If you click 'Accept all', we and our partners..."
1,"{'id': None, 'name': 'Yahoo Entertainment'}",Will Shanklin,The makers of Palworld have formed a new compa...,The maker of Xbox Game Pass stalwart Palworld ...,https://consent.yahoo.com/v2/collectConsent?se...,None,2024-07-10T18:48:24Z,"If you click 'Accept all', we and our partners..."
2,"{'id': None, 'name': 'Yahoo Entertainment'}",Cheyenne MacDonald,What to read this weekend: Keanu Reeves wrote ...,"New releases in fiction, nonfiction and comics...",https://consent.yahoo.com/v2/collectConsent?se...,None,2024-07-27T17:39:09Z,"If you click 'Accept all', we and our partners..."
3,"{'id': None, 'name': 'Gizmodo.com'}",Gordon Jackson,Terminator: Zero Debuts its First Trailer Just...,Netflix has released a trailer for its new Ter...,https://gizmodo.com/terminator-zero-anime-trai...,https://gizmodo.com/app/uploads/2024/07/unname...,2024-07-15T14:15:01Z,Netflix has released a trailer for its new Ter...
4,"{'id': None, 'name': 'Gizmodo.com'}",Cheryl Eddy,First Trailer for Rick and Morty: The Anime Re...,Adult Swim's new 10-episode anime series based...,https://gizmodo.com/first-trailer-for-rick-and...,https://gizmodo.com/app/uploads/2024/07/Screen...,2024-07-11T18:45:32Z,Rick and Morty fans are used to seeing the sho...


In [7]:
df.insert(0, 'source_id', pd.json_normalize(df['source']).get('id'))
df.insert(1, 'source_name', pd.json_normalize(df['source']).get('name'))

In [8]:
df = df.drop('source', axis=1)

In [9]:
df.head(5)

,source_id,source_name,author,title,description,url,urlToImage,publishedAt,content
0,None,Yahoo Entertainment,Cristina Alexander,Overwatch 2 shows off Transformers skins in an...,Blizzard and Hasbro have revealed the Transfor...,https://consent.yahoo.com/v2/collectConsent?se...,None,2024-07-08T22:23:59Z,"If you click 'Accept all', we and our partners..."
1,None,Yahoo Entertainment,Will Shanklin,The makers of Palworld have formed a new compa...,The maker of Xbox Game Pass stalwart Palworld ...,https://consent.yahoo.com/v2/collectConsent?se...,None,2024-07-10T18:48:24Z,"If you click 'Accept all', we and our partners..."
2,None,Yahoo Entertainment,Cheyenne MacDonald,What to read this weekend: Keanu Reeves wrote ...,"New releases in fiction, nonfiction and comics...",https://consent.yahoo.com/v2/collectConsent?se...,None,2024-07-27T17:39:09Z,"If you click 'Accept all', we and our partners..."
3,None,Gizmodo.com,Gordon Jackson,Terminator: Zero Debuts its First Trailer Just...,Netflix has released a trailer for its new Ter...,https://gizmodo.com/terminator-zero-anime-trai...,https://gizmodo.com/app/uploads/2024/07/unname...,2024-07-15T14:15:01Z,Netflix has released a trailer for its new Ter...
4,None,Gizmodo.com,Cheryl Eddy,First Trailer for Rick and Morty: The Anime Re...,Adult Swim's new 10-episode anime series based...,https://gizmodo.com/first-trailer-for-rick-and...,https://gizmodo.com/app/uploads/2024/07/Screen...,2024-07-11T18:45:32Z,Rick and Morty fans are used to seeing the sho...


In [10]:
df.to_csv('Data.csv')

# Snowflake Connection

### Using json file method and environment variable method

In [11]:
def snowflake_credentials(sf_config):
    try:
        sf_file_open = open(sf_config, 'r').read()
        sf_file_open = json.loads(sf_file_open)
    except:
        print('Error in file path')

    sf_connect = connect(
        # user = sf_file_open['user'],
        # password = sf_file_open['password'],
        
        user = os.environ.get('sf_user'),
        password = os.environ.get('sf_password'),
        account = sf_file_open['account'],
        role = sf_file_open['role'],
        warehouse = sf_file_open['warehouse'],
        database = sf_file_open['database'],
        schema = sf_file_open['schema']
    )
    
    # sf_connect = connect(
    #     user = sf_file_open.get('user'),
    #     password = sf_file_open.get('password'),
    #     account = sf_file_open.get('account'),
    #     role = sf_file_open.get('role'),
    #     warehouse = sf_file_open.get('warehouse'),
    #     database = sf_file_open.get('database'),
    #     schema = sf_file_open.get('schema')
    # )
    return sf_connect

In [12]:
# using all the credentials from a json file
# write_pandas(snowflake_credentials(r'C:\Users\JayashreeArunan\Desktop\Python\Data_extraction\configuration\sf_config.json'), df, 'NEW_TABLE', auto_create_table=True, overwrite=True)

In [13]:
# using the username and password stored in the system environment varibles and rest of the credentials from the json file
# write_pandas(snowflake_credentials(r'C:\Users\JayashreeArunan\Desktop\Python\Data_extraction\configuration\sf_config.json'), df, 'NEW_TABLE1', auto_create_table=True, overwrite=True)

### Using .env method

In [14]:
load_dotenv()

True

In [15]:
def snowflake_credentials():
    try:
        sf_connect = connect(
            user = os.getenv('user'),
            password = os.getenv('password'),
            account = os.getenv('account'),
            role = os.getenv('role'),
            warehouse = os.getenv('warehouse'),
            database = os.getenv('database'),
            schema = os.getenv('schema')
        )
        print('Connection successful')
    except:
        print('Error in credentials')
    return sf_connect

In [16]:
write_pandas(snowflake_credentials(), df, 'NEW_TRY', auto_create_table=True, overwrite=True)

Connection successful


(True,
 1,
 100,
 [('jwaypiiijk/file0.txt', 'LOADED', 100, 100, 1, 0, None, None, None, None)])